1. Клонирование репозитория и переход в рабочую директорию проекта

In [3]:
!git clone https://github.com/chirkoov/sprint2.git
%cd sprint2
!ls

fatal: destination path 'sprint2' already exists and is not an empty directory.
/home/ubuntu/sprint2
configs  requirements.txt  solution.ipynb  src


/home/ubuntu/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


2. Установка зависимостей

команду !sed -i '/pywin32/d' requirements.txt нужно закомментировать, если проект выполняется на windows

In [4]:
!sed -i '/pywin32/d' requirements.txt
!pip -q install -r requirements.txt

3. Загрузка исходного датасета

- создаем папку data внутри проекта
- скачиваем датасет

In [5]:
!mkdir -p data
!curl -L https://code.s3.yandex.net/deep-learning/tweets.txt -o data/raw_dataset.csv
!wc -l data/raw_dataset.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0  29.0M      0  0:00:03  0:00:03 --:--:-- 29.0M
1600498 data/raw_dataset.csv


4. Подготовка данных

Последовательно выполняется 3 этапа:

1. Очистка текста (`preprocess_dataset`)
   - удаление ссылок
   - приведение к нижнему регистру
   - удаление лишних символов

2. Токенизация (`tokenize_dataset`)
   - преобразование текста в `input_ids` с помощью токенизатора

3. Разделение на выборки (`split_dataset`)
   - 80% — train
   - 10% — validation
   - 10% — test

В результате формируются CSV-файлы для обучения и оценки модели.

In [6]:
from src.data_utils import preprocess_dataset, tokenize_dataset, split_dataset

preprocess_dataset("data/raw_dataset.csv", "data/dataset_processed.csv")
tokenize_dataset("data/dataset_processed.csv", "data/tokenized_dataset.csv")
split_dataset("data/tokenized_dataset.csv", "data/train.csv", "data/val.csv", "data/test.csv")

/home/ubuntu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


5. Проверка доступности GPU

In [7]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


CUDA available: True
GPU: Tesla T4


6. Обучения LSTM-модели

In [8]:
!python -m src.lstm_train

Using device: cuda
Epoch 1/10 | Train Loss: 5.4619 | Val Loss: 5.1952 | Rouge1: 0.0999 | Rouge2: 0.0129
Saved best model to models/lstm_next_token.pt
Epoch 2/10 | Train Loss: 5.2314 | Val Loss: 5.1409 | Rouge1: 0.0915 | Rouge2: 0.0117
Saved best model to models/lstm_next_token.pt
Epoch 3/10 | Train Loss: 5.1798 | Val Loss: 5.1197 | Rouge1: 0.1012 | Rouge2: 0.0129
Saved best model to models/lstm_next_token.pt
Epoch 4/10 | Train Loss: 5.1535 | Val Loss: 5.1121 | Rouge1: 0.1032 | Rouge2: 0.0132
Saved best model to models/lstm_next_token.pt
Epoch 5/10 | Train Loss: 5.1367 | Val Loss: 5.1053 | Rouge1: 0.1045 | Rouge2: 0.0129
Saved best model to models/lstm_next_token.pt
Epoch 6/10 | Train Loss: 5.1259 | Val Loss: 5.1038 | Rouge1: 0.0641 | Rouge2: 0.0085
Saved best model to models/lstm_next_token.pt
Epoch 7/10 | Train Loss: 5.1151 | Val Loss: 5.0981 | Rouge1: 0.0754 | Rouge2: 0.0104
Saved best model to models/lstm_next_token.pt
Epoch 8/10 | Train Loss: 5.1076 | Val Loss: 5.0950 | Rouge1: 0.0

7. Оценка LSTM-модели

- вычисляется значение функции потерь
- проводится генерация продолжений текста
- рассчитываются метрики ROUGE

In [9]:
!python -m src.eval_lstm

Evaluating Rouge: 100%|███████████████████████| 626/626 [05:45<00:00,  1.81it/s]
Val Loss: 5.0881 | Rouge1: 0.0832 | Rouge2: 0.0113
Examples:

        Example 1
Prompt:    armano became unemployed on may 11 took a little time off now looking for a job in bos or
Reference: pdx and thats the story
Generated: not


        Example 2
Prompt:    sad that the sun is
Reference: gone
Generated: over


        Example 3
Prompt:    yay thanks matcl
Reference: ##ayton
Generated: ##ub



8. Сравнение результатов с Transformer-моделью distilgpt2

In [10]:
!python -m src.eval_transformer

config.json: 100%|█████████████████████████████| 762/762 [00:00<00:00, 2.82MB/s]
tokenizer_config.json: 100%|██████████████████| 26.0/26.0 [00:00<00:00, 104kB/s]
vocab.json: 100%|██████████████████████████| 1.04M/1.04M [00:00<00:00, 3.83MB/s]
merges.txt: 100%|████████████████████████████| 456k/456k [00:00<00:00, 1.43MB/s]
tokenizer.json: 100%|██████████████████████| 1.36M/1.36M [00:00<00:00, 11.3MB/s]
model.safetensors: 100%|██████████████████████| 353M/353M [00:01<00:00, 177MB/s]
Loading weights: 100%|█| 76/76 [00:00<00:00, 2090.34it/s, Materializing param=tr
Processing: 100%|█████████████████████| 160064/160064 [1:03:26<00:00, 42.05it/s]
Rouge1: 0.0547 | Rouge2: 0.0051
Examples:

        Example 1
Prompt:    armano became unemployed on may 11 took a little time off now looking for a job in bos
Reference:  or pdx and thats the story
Generated: om, but he is still in


        Example 2
Prompt:    sad that the sun
Reference:  is gone
Generated: 's rays


        Example 3
Prompt:    ch

ВЫВОД:

По полученным метрикам ROUGE модель LSTM показывает лучшие результаты по сравнению с Transformer. Это означает, что её предсказания чаще совпадают с эталонными продолжениями текста.

С учётом требований мобильных устройств (ограничения по памяти и скорости), LSTM является предпочтительным вариантом, чем трансформер из-за более простой архитектуры и меньшим вычислительным затратам.

При этом примеры показывают, что предобученный Transformer (distilgpt2) генерирует более длинные и более логические продолжения. Однако его использование требует больше ресурсов. Потенциальное улучшение качества LSTM возможно за счёт увеличения объёма данных и более длительного обучения.